# Enviar job de Vertex
Enviar job de vertex para el entrenamiento del modelo en cloud

## Consideraciones generales importantes Vertex AI - Noviembre 2023
- El job de entrenamiento que se envia se guarda en el menu principal **"Model Development"**, específicamente en el submenú **"Entrenamiento"**

- Por otro lado, el modelo que queda entrenado queda registrado en el menu principal **"Deploy and Use"**, específicmente en el submenú **"Registro de Modelos"** (solo si se logro ejecutar bien el job y entrenó el modelo. Este funciona como un repositorio de modedlos). Luego de tener registrado el modelo, si se desea, se puede deployar en un endpoint (para realizar predicciones en línea con un delay muy pequeño)  y el endpoint queda registrado en el menu **"EndPoint"**

## Consideraciones para el entrenamiento realizado en este notebook
- Se utiliza la clase **CustomTrainingJob**. Documentación: https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomTrainingJob#methods
- **En este ejemplo, se ENVIA UN SCRIPT CON EL CÓDIGO DE ENTRENAMIENTO A GCP.**


- **DOCUMENTACIÓN PYTHON DE LA LIBRERÍA COMPLETA AIPLATFORM**, INCLUYE ENTRE OTROS EL **CustomTrainingJob** QUE SE UTILIZA PARA ENVIAR UN SCRIPT DE ENTRENAMIENTO A LA NUBE, el que se utiliza más adelante en este ejemplo https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform

- **Repo Github oficial de Vertex AI - ejemplos interesantes**:
- https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/get_started_with_vertex_distributed_training.ipynb
- https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/hyperparameter_tuning_xgboost.ipynb

In [ ]:
import pandas as pd
import datetime as dt
import numpy as np

In [ ]:
import sys
print(sys.prefix)

In [ ]:
from google.cloud import aiplatform
from google.cloud.aiplatform import gapic as aip

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_ID_DS = os.environ.get("PROJECT_GCP", "")

### Paso 0. Parámetros generales

In [ ]:
### PARÁMETROS GENERALES GCP ###

# definir el proyecto. Diría que se tiene que hacer todo en un único proyecto porque sino vertex como que no funciona (creo)
PROJECT_ID = PROJECT_ID_DS

# definir un bucket (ya creado) para guardar los archivos que genera el usar VERTEX AI.
# No necesariamente puede ser un bucket, puede ser una carpeta dentro de un bucket. Lo cual es mejor si se quiere tener un
# bucket para un caso de uso y dentro de ese bucket un folder dedicado a todo los códigos de VERTEX AI
BUCKET_ID = '{bucket_name}/vertex-ai'

# definir una región, VERTEX SE INICIALIZA EN UNA REGIÓN y se utiliza algún recursos de vertex ej: datesets deben de estar 
# definidos en la misma región que se inicializó el código. Importante mayo 2023: dataset no están implementados para todas las regiones
REGION = '{region}'

In [ ]:
### PARÁMETROS GENERALES EJECUCIÓN ###

# obtener la hora actual de cuándo se comenzó la ejecución - hash
now = dt.datetime.now()
date_time = now.strftime("%Y_%m_%d_%H_%M_%S")


# identificacion del tipo de caso de uso (y también tipo de modelo) que se va a usar poara registrar el entrenamiento
identity_kind_use_case = 'basic_job_vertex'  

In [ ]:
print('Parámetros Generales GCP')
print('PROJECT_ID: ', PROJECT_ID)
print('BUCKET_ID: ', BUCKET_ID)
print('REGION: ', REGION)

print('\n----')
print('Parámetros Específicos job entrenamiento')
print('date_time: ', date_time)
print('identity_kind_use_case: ', identity_kind_use_case)

### Paso 1. Crear script de entrenamiento

### Información importante
- Enviar un job de entrenamiento utilizando la función **"CustomTrainingJob"** que **envia un script de python para ser corrido en la nube**. No acepta carpetas con multiples scripts.


- El script de entrenamiento puede tener también escrita la limpieza/transformación de los datos, no hay problema en eso (se le pueden pasar los datos ya limpios llegar y entrenar o se le puedan pasar los datos sucios y que los limpie primero y luego entrene el modelo). El script de entrenamiento soporta todo el código que se desee agregar (solo la consideración mencionada al inicio, tiene que ser SOLO UN SCRIPT)


- **El script que se le pasa al job de entrenamiento puede recibir argumentos.** Los valores de los argumentos del script se le pasan a vertex AI cuando se envía el job para correr el script y entrenar el modelo.


- Los argumentos que se le pasan al script del job de entrenamiento puede ser la típica variable (ej: batch_size = 14), así como también strings los cuales pueden ser el path a algún archivo de GCS por ejemplo (Ej: pasar como argumento el path de un json de GCS, donde este json tiene múltiples parámetros para ser usados en el script.py). **ENTONCES, si se le quieren pasar múltiples argumentos y tiene que definirse 1 por 1 haciendo un listado gigante de argumentos, se le puede pasar un solo argumento con el path a un json de gcs, el cual contiene todos los argumentos de entrenamiento** (luego ese json con todos los parámetros puede ser modificado de acuerdo a lo que se necesite y el código para entrenar el modelo permanece sin cambios).


- El script.py que se crea no necesita un método __init__, basta con escribir el código y listo.


- El script de entrenamiento para cloud, NO TIENE NINGUNA DIFERENCIA CON UN SCRIPT DE ENTRENAMIENTO QUE SE CORRA LOCAL. Es decir, **si tengo un script de entrenamiento que corro localmente, puedo ese mismo script enviarlo a VERTEX AI sin modificarlo nada y listo, modelo entrenado en cloud**


- **MUY IMPORTANTE** CUANDO SE CORRE EL SCRIPT DE ENTRENAMIENTO, ENTRENA EL MODELO, PERO NO LO REGISTRA EN EL MENU "MODELOS" DE FORMA AUTOMÁTICA. PARA QUE EL **MODELO QUEDE REGISTRADO EN EL MENU "MODELOS" (y después usarse para predecir batch o implementarlo para predecir en línea) SE NECESITA GUARDAR EL MODELO EN GCS**, PERO ADEMÁS, SE NECESITA GUARDAR EN UN PATH EN ESPECÍFICO DEFINIDO COMO UNA VARIABLE GLOBAL **"AIP_MODEL_DIR"**. Además el **artefacto con el modelo debe quedar guardado en GCS con el nombre "model.pkl" SÍ O SÍ**. Pero, el problema es que si no se desea registrar el modelo, se omite la parte de guardar, el código corre con normalidad pero POR DEFECTO VERTEX VA A BUSCAR EL MODELO PARA REGISTRARLO Y ESTE AL NO EXISTIR (no haberse guardado) TIRA ERROR EN EL ENTRENAMIENTO, lo que dificulta un poco más el proceso de debugging cuando falla algo


- Ejemplo del path por defecto **AIP_MODEL_DIR**: gs://{bucket-name}/aiplatform-custom-training-2022-05-01-13:27:34.025/model/. Donde "dicovery-vertex-ai" es el bucket definido al inicializar la libreria ai platform de vertex. El resto se crea de forma automática. Cuando se guarda el artefacto del modelo, el path queda de la siguiente forma: **gs://{bucket-name}/aiplatform-custom-training-2022-05-01-13:27:34.025/model/model.pkl**


- Importante, por defecto **VERTEX SIEMPRE NECESITA QUE EL PATH AL ARTEFACTO DEL MODELO SEA DE LA FORMA ".../model/model.pkl"** Esto quiere decir que siempre debe existir una carpeta model y dentro de esta un artefacto model.pkl. Además, lo ideal es que existe una carpeta para identificar el caso de uso y un hash para identificar la ejecución. Así el **path de guardado queda de la forma "gs://{bucket}/{use_case + hash}/model/model.pkl"** Un path de este estilo es creado por defecto en las ejecuciones de los jobs si no se toca la variable de ambiente **"AIP_MODEL_DIR"** en el script de entrenamiento y el argumento **base_output_dir** al momento de enviar el job


- De acuerdo a los valores de los parámetros (leer documentación), es posible modificar el path donde se va a guardar el modelo y que no sea el que se crea automáticamente que queda guardado en **"AIP_MODEL_DIR"**, sino en un path de GCS definido por el usuario. Lo importante es definir el argumento **base_output_dir** con el path donde vertex va a buscar el modelo para registrarlo, y luego al guardar el artefacto en GCS utilizar ese mismo path; y no el automático. **MUY IMPORTANTE el path que se define en base_output_dir debe ser de la forma "gs://{bucket}/{use_case + hash}/"**, es decir debe omitirse del path ".../model/model.pkl" ya que es lo obligatorio de vertex. Obviamente al guardar el artefacto del modelo debe hacerse en la ruta completa "gs://{bucket}/{use_case + hash}/model/model.pkl"


- **Documentación variable ambiente** (ej AIP_MODEL_DIR donde se guarda el artefacto del modelo): https://cloud.google.com/vertex-ai/docs/training/code-requirements#environment-variables
- **Documentación guardar modelos vertex ai (entrenando en vertex ai)(guardar el pkl resultante en un cierto path, para que customtrainingjob lo reconozca)**: https://cloud.google.com/vertex-ai/docs/training/exporting-model-artifacts?hl=es-419#scikit-learn
- **Documentación clase CustomTrainingJob**: https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomTrainingJob#methods
- **Repo Github oficial de Vertex AI - ejemplo interesante**: https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/get_started_with_vertex_distributed_training.ipynb

In [ ]:
# ESCRIBIR EL SCRIPT DE ENTRENAMIENTO.

In [ ]:
# PROBAR CORRER EL SCRIPT QUE GENERÉ, PARA VER SI FUNCIONA (lo ideal es probar con un dataset pequeño)
# %run train_model.py

### Paso 2: Inicializar Vertex AI
- Antes de poder enviar un job de entrenamiento o realizar cualquier operación en VERTEX AI es necesario inicializarlo.


- Parámetros necesarios para inicializar:
    - project: El projecto de GCP
    - location: La región en la que se va a ejecutar el entrenamiento
    - staging_bucket: el bucket de GCS donde se guardan los artefactos que generan los jobs de entrenamiento de vertex


- Documentación: https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_init


- Total de parámetros que se le puede pasar al método init (ver documentación): project, location, experiment (no sé que es), experiment_description, staging_bucket, credentials, ncryption_spec_key_name

In [ ]:
aiplatform.init(project = PROJECT_ID, location = REGION, staging_bucket = BUCKET_ID)

### Paso 3. Definir parámetros necesarios para CREAR la instancia del job de entrenamiento (aún no se envia)

- Para entrenar un modelo en cloud se deben realizar 2 pasos: el primero crear la instancia de la clase del entrenamiento (CustomTrainingJob) y en segundo lugar enviar el job de entrenamiento (método de la instancia)

---------------------------
- Para crear la instancia de la clase del job de entrenamiento, se necesitan los siguientes parámetros:
    - **display_name**: El nombre del job de entrenamiento. Con este nombre queda registrado el job de entrenamiento
    - **script_path**: El nombre del script.py que se envia para entrenamiento
    - **container_uri**: El path con la imagen docker para el entrenamiento. Puede ser de las facilitadas por google o una propia. Recomendable utilizar una de google y solo instalar los packages faltantes en esa instancia. Crear una propia desde cero es un problema.
    - **model_description**: Descripción que aparece en el menú "modelos" del modelo que se registre
    - **requirements**: Requisitos de packages a instalar en la imagen pasada en container_uri
    - **model_serving_container_image_uri**: Path con la imagen docker para las prediciones. Facilitad por google o una propia


---------------------------
- INFO containers base de google:
    - **Container para el entrenamiento**: https://cloud.google.com/vertex-ai/docs/training/pre-built-containers
    - **Container para la predicción**: https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers

---------------------------
- INFO, LOS ÚNICOS PARÁMETROS QUE SON OBLIGATORIOS SON:
    - **display_name**
    - **script_path**
    - **container_uri**

---------------------------
- Documetación de la clase **"CustomTrainingJob"** (se usa para crear la instancia y luego correr el método run para enviar a su entrenamiento): https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomTrainingJob


- **IMPORTANTÍSIMO mayo 2023: Los contenedores prebuild de "sklearn" están muy desactualizados. Por lo tanto lo ideal es utilizar los container de "tensorflow" ya que estos containers prebuild que GCP actualiza y agregar en los requirements los package que faltan. El container prebuild de tensorflow tiene las mismas versiones que los notebooks de workbench que tiene Vertex así entrenar un modelo en el workbench es compatible con enviar el job de entrenamiento**

In [ ]:
### definir el nombre del job que se enviará. Algo que indentifique de qué es el job + hora envio ###
job_name = identity_kind_use_case + '__job_train__' + date_time
job_name

In [ ]:
# ### definir el contrainer para el ENTRENAMIENTO y para LA PREDICCIÓN - facilitados por google ####
# container_train = 'us-docker.pkg.dev/vertex-ai/training/scikit-learn-cpu.0-23:latest'
# container_deploy = 'us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-23:latest'

In [ ]:
# definir container para el ENTRENAMIENTO y predicción online - UTILIZAR LOS ÚLTIMOS DE TF e instalar packages faltantes. NOV 2023
container_train = 'us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-12.py310:latest'
container_deploy = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-12:latest'

In [ ]:
### definir el path al script de entrenamiento ###
path_train_script = 'train_model.py'

In [ ]:
### definir la descripción del modelo ###
description = 'entrenar modelo basico vertex AI'

In [ ]:
### definir los requirements ###

# el db-types es necesario para poder consultar tablas de bigquery
list_requirements = ["google-cloud-bigquery>=2.20.0", "db-dtypes", "pandas==2.0.3", "numpy==1.23.5", "scikit-learn==1.3.1"]

### Paso 4. Definir parámetros necesarios para ENVIAR job de entrenamiento - usando CPU

- The run function creates a training pipeline that trains and creates a Model object. After the training pipeline completes, the run function returns the Model object.


- La istancia de la clase "CustomTrainingJob" definida como "job" se le llama el método **"run" "job.run(xxx)"** y se envía el job de entrenamiento a VERTEX AI. **Al llamar al método "run" se puede hacer SIN PASARLE NADA o PASÁNDOLE ARGUMENTOS (opcionales)**


- Algunos de los ARGUMENTOS que se le pueden pasar al método **"run"** son los siguientes:

    - model_display_name: The display name of the Model if the script produces a managed Model (Si no se le da un nombre al modelo, se emplea el mismo nombre que se le dio al job y se le agrega el sufijo "-model")
    - machine_type: The type of machine to use for training. Por defecto utiliza: "n1-standard-4"
    - **args**: The command line arguments to be passed to the Python script. Se pasa una lista con los args. Ejemplo 1: ["--epochs=5", "--batch_size=16", "--distribute=multiworker"]. Ejemplo 2: ["--model-dir=" + MODEL_DIR, "--epochs=5"]
    - replica_count: The number of worker replicas.
    - accelerator_type: The hardware accelerator type. (Parámetro GPU: qué gpu utilizar par entrenamiento)
    - accelerator_count: The number of accelerators to attach to a worker replica. (Parámetro GPU: cantidad de gpu a utilizar)
    - **base_output_dir**: **GCS output directory of job**. Path de GCS donde se dice que está guardado el modelo y que se utiliza para registrarlo en el submenú "models", **si no se le pone nada utiliza el valor por defecto que es la variable de entorno 'AIP_MODEL_DIR'**


- **Ningún argumento es obligatorio (todos tienen sus valores por defecto). Se completan de acuerdo a la necesidad**

In [ ]:
### definir el nombre con el que queda registrado (en VERTEX AI) el modelo resultado del entrenamiento ###
# De qué es el modelo +  hora de envio
model_name = identity_kind_use_case  + '__model__' + date_time 
model_name

In [ ]:
### definir el tipo de máquina para hacer el entrenamiento ###

machine_type_train = "n1-standard"
vcpu_train = "4"
train_compute = machine_type_train + "-" + vcpu_train

print("Train machine type: ", train_compute)

### Paso 5. Crear instancia del job de entrenamiento a VERTEX AI (CustomTrainingJob)
- Define your custom TrainingPipeline on Vertex AI.


- Use the **CustomTrainingJob class** to define the TrainingPipeline.


- **IMPORTANTE, AÚN NO SE ENTRENADA NADA NI SE CREA NADA EN VERTEX**: 

    - Cuando se crea la instancia "job" de la clase "CustomTrainingJob" aún no aparece nada registrado en el menú "Entrenamiento" de VERTEX AI
    
    - De la instancia creada, definida por la variable "job", **aún no se le pueda llamar ningún atributo por ejemplo **"job.resource_name" porque aún no se crea nada en VERTEX AI**. Por lo tanto si llamara "job.resource_name" u otro atributo retornaría el siguiente error "CustomTrainingJob resource has not been created"; para poder llamar estos atributos es necesario enviar el job creado a ser entrenado "job.run()"

In [ ]:
# PRIMERO SE LLAMA UNA INSTANCIA DE LA CLASE
job = aiplatform.CustomTrainingJob(
    display_name = job_name,
    script_path = path_train_script,
    model_description = description,
    container_uri = container_train,
    requirements = list_requirements,
    model_serving_container_image_uri = container_deploy,
)

In [ ]:
job

### Paso 6. Enviar el job de entrenamiento a VERTEX AI (CustomTrainingJob)

- Use the run function to start training

- Luego de ser enviado el job de entrenamiento y este terminado correctamente aparecen **3 registros en vertex AI**. El primero en el **menú "models"** donde está el modelo entrenado y otros dos en **menu entrenamiento**, uno en el sub menú **"entrenamiento/trainning pipelines"** y otro en el sub menu **"entrenamiento/custom jobs"**


- Importante, **al script de entrenamiento se le puede poner PRINTS, los cuales al abrir el job de entrenamiento en vertex (menu entrenamiento/custom jobs), aparece la opción de ir al logging/registros**, y de ahí ver los loggins del job enviado (el cual incluye los print)


- Importante 2: Al **enviar el job de entrenamiento a VERTEX AI se guardan dos archivos en el bucket de GCS definido al inicializar Vertex AI**
    - Un archivo **comprimido .tar.gz** donde se guarda el script que se envió al entrenamiento. Para identificar el archivo se le añade la fecha de forma automática en que este se corrió (creo que no se puede modificar el nombre desde parámetros). Se guarda por defecto en el bucket definido al inicializar vertex.
    - Una **carpeta donde se guarda el artefacto del modelo**. Para identificar la carpeta se añade la fecha de forma automática en que esta es creada. El path a esta carpeta se guarda en la variable de ambiente "AIP_MODEL_DIR" (este path si se puede modificar)
    - OBS: tanto el archivo compromido como la carpeta tiene una fecha, PERO ESTA ES DISTINTA EN CADA UNO DE LOS ARCHIVOS, porque estos artefactos NO se crean al mismo tiempo
    
    
- Importante 2.b. La ubicación del compromido con los paquetes y la carpeta con el modelo se pueden ver en el menu "entrenamiento" seleccionado el job que se desea revisar y posteriormente clickeando en "detalles de la versión"
  

- Importante 3. En el menu "modelos" uno puede elegir el modelo y lo envia a un menu con las diferentes versiones de ese modelo, y al hacer click en el ID de la versión, me envía a un menú más detallado del modelo (si quiero implementarlo, hacer predicciones batch, detalles de la versión). El cual es el mismo menú que aparece si uno va a "entrenamiento/training pipelines" elige el job de entrenamiento y reenvia al menú más detallado del modelo. Es decir, **se puede acceder al menu detallado del modelo ya sea desde el menu "models" y buscando el modelo y su versión específica o desde el menu "entrenamiento" y seleccionando el pipeline job que generó dicho modelo**

- Importante 4. Para que **no aparesca el texto que está corriendo el job y se pueda seguir utilizando el código se puede utilizar el parámetro "sync" y setear en "False"**

In [ ]:
model = job.run(
    model_display_name = model_name,
    replica_count = 1,
    machine_type = train_compute,
    sync = True
)

In [ ]:
model